In [27]:
import numpy as np
import pandas as pd
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils, print_summary
import keras.backend as K
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint


In [28]:
data = pd.read_csv("data.csv")
dataset = np.array(data)
np.random.shuffle(dataset)
X = dataset
Y = dataset
X = X[:, 0:1024]
Y = Y[:, 1024]
X_train = X[0:70000, :]
X_train = X_train / 255.
X_test = X[70000:72001, :]
X_test = X_test / 255.
   # Reshape
Y = Y.reshape(Y.shape[0], 1)
Y_train = Y[0:70000, :]
Y_train = Y_train.T
Y_test = Y[70000:72001, :]
Y_test = Y_test.T

In [29]:
print("number of training samples = " + str(X_train.shape[0]))
print("number of test samples = " + str(X_test.shape[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))

number of training samples = 70000
number of test samples = 2000
X_train shape: (70000, 1024)
Y_train shape: (1, 70000)
X_test shape: (2000, 1024)
Y_test shape: (1, 2000)


In [19]:
image_x = 32 #height 
image_y = 32  #width
train_y = np_utils.to_categorical(Y_train) #mporting from keras 
test_y = np_utils.to_categorical(Y_test)
train_y = train_y.reshape(train_y.shape[1], train_y.shape[2])
test_y = test_y.reshape(test_y.shape[1], test_y.shape[2])
X_train = X_train.reshape(X_train.shape[0], image_x, image_y, 1)

In [20]:
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(train_y.shape))

X_train shape: (70000, 32, 32, 1)
Y_train shape: (70000, 37)


In [21]:
# Buliding the sequential model that is output is stored in h5 file
def keras_model(image_x,image_y):
    num_of_classes = 37  #there are 35 classfied consonants letters in the hindi letters and two classes are for check
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5, 5), input_shape=(image_x, image_y, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
    model.add(Flatten())
    model.add(Dense(num_of_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath = "hindi_model.h5"
    checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint1]

    return model, callbacks_list

In [26]:
X_test = X_test.reshape(X_test.shape[0], 32, 32, 1)
model, callbacks_list = keras_model(image_x, image_y)
model.fit(X_train, train_y, validation_data=(X_test, test_y), epochs=4, batch_size=64,callbacks=callbacks_list)
scores = model.evaluate(X_test, test_y, verbose=0)
print("Convolutional neural network Error: %.2f%%" % (100 - scores[1] * 100))
print_summary(model)
model.save('hindi_model.h5')


Train on 70000 samples, validate on 2000 samples
Epoch 1/4
70000/70000 [==============================] - 1118s 16ms/step - loss: 0.7491 - acc: 0.7863 - val_loss: 0.2846 - val_acc: 0.9080

Epoch 00001: val_acc improved from -inf to 0.90800, saving model to hindi_model.h5
Epoch 2/4
70000/70000 [==============================] - 1085s 16ms/step - loss: 0.2316 - acc: 0.9303 - val_loss: 0.1782 - val_acc: 0.9385

Epoch 00002: val_acc improved from 0.90800 to 0.93850, saving model to hindi_model.h5
Epoch 3/4
70000/70000 [==============================] - 1126s 16ms/step - loss: 0.1515 - acc: 0.9539 - val_loss: 0.1393 - val_acc: 0.9555

Epoch 00003: val_acc improved from 0.93850 to 0.95550, saving model to hindi_model.h5
Epoch 4/4
70000/70000 [==============================] - 1078s 15ms/step - loss: 0.1098 - acc: 0.9668 - val_loss: 0.1143 - val_acc: 0.9650

Epoch 00004: val_acc improved from 0.95550 to 0.96500, saving model to hindi_model.h5
Convolutional neural network Error: 3.50%
________